<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span></li><li><span><a href="#Финальная-проверка-модели" data-toc-modified-id="Финальная-проверка-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Финальная проверка модели</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [ ]:
# import libraries
!pip install optuna --quiet --disable-pip-version-check
import optuna
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier

In [ ]:
# technical aspects
tqdm.pandas()
lemmatizer = nltk.stem.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))



<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты и настройки на месте
</div>


In [ ]:
# load dataset
try:
    df = df.read_csv('toxic_comments.csv', index_col=[0])
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col=[0])

In [ ]:
df # show data

кривые индексы

In [ ]:
# reset index & 'toxic' distribution
df = df.reset_index(drop=True)
df['toxic'].value_counts()

Видим дисбаланс классов

In [ ]:
# preprocessing function for text

def text_preprocessing(text):
    text = text.lower() # low case
    text = re.sub(r"[^a-z ]", ' ', text) # cleaning from signs
    text = text.split() # tokenization
    text = [lemmatizer.lemmatize(word) for word in text] # lemmatize
    text = [word for word in text if word not in stop_words] # cleaning from stopwords
    text = " ".join(text) # join tokens to text
    return text

In [ ]:
# apply func
tweets = df['text'].progress_apply(text_preprocessing)
tweets

In [ ]:
# split of samples

X_train, X_test, y_train, y_test = train_test_split(tweets, df.toxic, test_size=0.2, random_state=1234, shuffle=True)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1234, shuffle=True)

In [ ]:
# tf-idf vectors

count_tf_idf = TfidfVectorizer() # init tfidf
tf_idf_train = count_tf_idf.fit_transform(X_train) # fit on TRAIN data and transform to tf_idf
tf_idf_test = count_tf_idf.transform(X_test) # test data transform to tf_idf
# tf_idf_val = count_tf_idf.transform(X_val)

**Вывод:** В ходе подготовки было сделано:
* загружен датасет;
* предобработка текста:
   * приведение к нижнему регистру;
   * очистка от лишних символов, чисел;
   * токенизация;
   * лемматизация;
   * очистка от стоп слов.
* разделение данных на тренировочную и тестовую выборки;   
* преобразования текстов в вектора TF_IDF.

## Обучение моделей

Весь процесс обучения будет заложен происходить одновременно с подбором гиперпараметров на optuna. В финальный тест пойдет лучшая модель с лучшими гиперпараметрами.

В качестве моделей был сделан выбор в пользу линейных моделей(Логистическая регрессия, линейный метод опорных векторов и "пассивно агрессивный" классификатор), т.к. они работают с разреженными матрицами быстрее чем деревья и ансамбли.

В качестве гиперпараметра был выбран параметр "С" - величина обратная величине регуляризации. Больше никаких гиперпараметров не рассматривалось в пользу ускорения обучения.

Т.к классы не сбалансированы, то они взвешиваются при инициализации моделей.

Для проверки качества используется кросс-валидация. Берется среднее значение после каждого испытания.

Целевой функцией является F1-мера, нам нужно максимизировать ее значение, также необходимо получить значение F1 не менее 0.75

In [ ]:
# Define an objective function

def objective(trial):

    # choise hyperparameters(model, c-value)
    clf_name = trial.suggest_categorical('classifier', ['LogReg', 'LinearSVC', 'PassAgrClassifier'])
    if clf_name == 'LogReg':
        c_reg = trial.suggest_float('c_reg', 0.01, 10, log=True)
        clf_obj = LogisticRegression(C=c_reg, class_weight='balanced', max_iter=300, random_state=312, n_jobs=-1)
    elif clf_name == 'LinearSVC':
        c_svc = trial.suggest_float('c_svc', 0.01, 1, log=True)
        clf_obj = LinearSVC(C=c_svc, class_weight='balanced', random_state=312)
    else:
        c_pac = trial.suggest_float('c_pac', 0.01, 1, log=True)
        clf_obj = PassiveAggressiveClassifier(C=c_pac, class_weight='balanced', random_state=312, n_jobs=-1)

    # calculate obj_val
    f1 = cross_val_score(clf_obj, tf_idf_train, y_train, scoring='f1', cv=4, n_jobs=-1).mean()

    return f1 # objective value

In [ ]:
study = optuna.create_study(direction='maximize')  # create a new study
study.optimize(objective, n_trials=20, n_jobs=-1)  # invoke optimization of the objective function

In [ ]:
# plot results
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

F1 метрика выше 0.75 а значит цель достигнута, осталось протестировать модель.

In [ ]:
study.best_params # get best params

{'classifier': 'LogReg', 'c_reg': 7.5308061074557795}

*p.s. Данные результаты были получены в результате прогона через 100 испытаний что заняло больше часа. Если нет времени ждать, то можно использовать их*

**Вывод:** 
* На этапе обучения моделей было сделан упор на линейные модели('LogReg', 'LinearSVC', 'PassAgrClassifier'). 
* С помощью фреймворка optuna былы подобраны гиперпараметры: модель, с-значение.
* Результаты испытаний визуализированы на графике
* Лучший результат показала логистическая регрессия с f1 показателем 0.762.

## Финальная проверка модели

In [ ]:
# best params to var
try:
    best_c = study.best_params['c_reg']
    best_model = study.best_params['classifier']
except:
    best_c = 7.5308061074557795
    best_model = 'LogReg'

In [ ]:
# init final model

if best_model == 'LogReg':
    final_model = LogisticRegression(max_iter=300, random_state=312, class_weight='balanced', C=best_c)

elif best_model == 'LinearSVC':
    final_model = LinearSVC(class_weight='balanced', random_state=312, C=best_c)

else:
    final_model = PassiveAggressiveClassifier(class_weight='balanced', random_state=312, C=best_c)

In [ ]:
# final test

final_model.fit(tf_idf_train, y_train)
preds = final_model.predict(tf_idf_test)

print('f1 = ', f1_score(y_test, preds),
      '\naccuracy = ', accuracy_score(y_test, preds),
      '\nprecision = ', precision_score(y_test, preds),
      '\nrecall = ', recall_score(y_test, preds),
      '\nroc_auc = ', roc_auc_score(y_test, preds))

На этапе тестирования f1 показало 0.767, что больше требуемого значения 0.75 а значит тестирование прошло успешно.

Также для наглядности вывел другие метрики качества, их значения подтверждают хорошее качество модели.

In [ ]:
# 'dummy' check

dummy_model = DummyClassifier(strategy='stratified')
dummy_model.fit(tf_idf_train, y_train)
preds = dummy_model.predict(tf_idf_test)

print('f1 = ', f1_score(y_test, preds),
      '\naccuracy = ', accuracy_score(y_test, preds),
      '\nprecision = ', precision_score(y_test, preds),
      '\nrecall = ', recall_score(y_test, preds),
      '\nroc_auc = ', roc_auc_score(y_test, preds))

Также модель прошла проверку на адекватность, показатели Dummy предиктора значительно ниже.

**Вывод:** 
* Финальная модель на тестировании показала f1=0.767 что соответствует целевому значению.
* Модель прошла проверку на адекватность. Метрики качества на порядок выше чем у бейзлайна.

## Выводы

**Цель:** построить модель для классификации токсичных комментариев с метрикой F1 не меньше 0.75.

Задачи:
* Загрузка и предобработка данных:
  * Загружен датасет toxic_comments.csv
  * Приведение текста к нижнему регистру
  * Очистка от лишних символов и чисел
  * Токенизация
  * Лемматизация
  * Удаление стоп-слов
  * Разделение выборки на обучающую и тестовую
  * Преобразование текстов в вектора TF-IDF
* Обучение и подбор гиперпараметров:
  * Использованы линейные модели: логистическая регрессия, LinearSVC, Passive Aggressive
  * С помощью Optuna подобраны гиперпараметры: модель и С-значение
  * Для оценки качества использована кросс-валидация
  * Целевая метрика - F1 (максимизация)
* Тестирование финальной модели:
  * Лучшая модель - логистическая регрессия
  * На тестовой выборке получено значение F1=0.767
  * Проведена проверка на адекватность с помощью бейзлайна

**Вывод:** цель и задачи проекта выполнены. Построена модель классификации токсичных комментариев с заданным качеством F1>0.75.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны